In [ ]:
%pylab
from scipy.fftpack import fft, ifft, fftshift, fftfreq

Let's check out the FFTs for C+A cos(ft + phi) and A exp(- B (t- L/2)^2)

In [14]:
N = 4096
L = 1.
t = np.linspace(0,1,N)
freqs = fftfreq(N) * N / L
f1 = 2. + 0.5 * np.cos(2*np.pi * 10 * t + np.pi/4) # cosine function
fft1 = fft(f1)
plt.scatter(freqs, np.abs(fft1), label="Cosine")
plt.xlim(-20, 20)
plt.xlabel("Frequency")
plt.ylabel(r"$\|f_k\|$")
plt.legend()
plt.savefig("cosine_fft.pdf", bbox_inches='tight')

plt.clf()
f2 = 0.5 / (2*np.pi*0.1**2)**0.5 * np.exp((t - L/2)**2 / 2 / (0.1**2)) # cosine function
fft2 = fft(f2)

plt.scatter(freqs, np.abs(fft2), label='Gaussian')
plt.xlim(-20, 20)
plt.xlabel("Frequency")
plt.ylabel(r"$\|f_k\|$")
plt.legend()
plt.savefig("gaussian_fft.pdf", bbox_inches='tight')